In [1]:
from basic_tools import *

debug=False

In [2]:
h2_myuni_sorted=read_ldsc('uni',False)
h2_myuni_sorted.head()

,uni.filename,uni.Total Observed scale h2,uni.Total Observed scale h2_se,uni.Total Liability scale h2,uni.Total Liability scale h2_se,uni.Lambda GC,uni.Mean Chi^2,uni.Intercept,uni.Intercept_se,uni.Ratio
phenotype,,,,,,,,,,
50_irnt,data/out_final/uni.50_irnt.ldsc.log,0.3772,0.0197,0.3772,0.0197,2.5074,4.7497,1.6108,0.0373,0.1629 (0.0099)
20153_irnt,data/out_final/uni.20153_irnt.ldsc.log,0.3293,0.0209,0.3293,0.0209,1.5696,2.1139,1.2127,0.0188,0.1909 (0.0169)
5097_irnt,data/out_final/uni.5097_irnt.ldsc.log,0.2878,0.0233,0.2878,0.0233,1.3169,1.5435,1.1186,0.0134,0.2183 (0.0246)
5098_irnt,data/out_final/uni.5098_irnt.ldsc.log,0.2825,0.0223,0.2825,0.0223,1.3203,1.5455,1.1236,0.0135,0.2266 (0.0247)
5133_irnt,data/out_final/uni.5133_irnt.ldsc.log,0.2771,0.0221,0.2771,0.0221,1.3068,1.5356,1.1237,0.0135,0.231 (0.0252)


In [3]:
h2_myuni_sorted.shape

(561, 10)

In [4]:
suffix_bp_list_power="bp300 bp128 bp64 bp32 bp16 bp8".split()
suffix_bp_list_full="bp300 bp128 bp64 bp32 bp16 bp8".split()

h2_bp_sorted_list=[]
for suffix in suffix_bp_list_full:
    h2_bp_sorted_list.append(read_ldsc(suffix,verbose=True))
    print(suffix,"was loaded")

bp300 was loaded
bp128 was loaded
bp64 was loaded
bp32 was loaded
bp16 was loaded
bp8 was loaded


In [6]:
h2_total=h2_bp_sorted_list[0].copy()
for h2_bp_sorted in h2_bp_sorted_list[1:]:
    h2_total=h2_total.join(h2_bp_sorted)

h2_total=h2_total.join(h2_myuni_sorted)

In [7]:
regression_result_bp_list_dict=dict([make_regression_result_list(h2_total,phenotype,suffix_bp_list_power) for phenotype in phenotypes_uni_filtered.index])

In [8]:
h2_bp8_df=pd.DataFrame([[phenotype]+(regression_result_list[-1]['h2']/regression_result_list[-1]['h2'].sum()).tolist() for phenotype,regression_result_list in regression_result_bp_list_dict.items()])
h2_bp8_df=h2_bp8_df.set_index(0)

In [9]:
edges=h2_bp8_df.T.corr().mask(np.tril(np.ones(h2_bp8_df.T.corr().shape)).astype(np.bool)).stack()
edges=pd.DataFrame([[edge[0][0],edge[0][1],edge[1]] for edge in edges.iteritems()],columns=['source','target','value'])

In [10]:
edges_filtered=edges.loc[(edges['value'] > 0.97) & (edges['source'] != edges['target'])]

In [11]:
groups=[]
for idx,row in edges_filtered.iterrows():
    try:
        group_index=[(row['source'] in groups[i] or row['target'] in groups[i]) for i in range(len(groups))].index(True)
        groups[group_index].add(row['source'])
        groups[group_index].add(row['target'])
    except:
        groups.append({row['source'],row['target']})

In [12]:
phenotypes_par_filtered=phenotypes_uni_filtered.copy()

In [13]:
phenotypes_par_filtered['group_id']=[np.argmax([False]+[idx in group for group in groups]) for idx,row in phenotypes_par_filtered.iterrows()]

In [14]:
phenotypes_par_filtered['uni.Total Liability scale h2']=h2_total['uni.Total Liability scale h2']
phenotypes_par_filtered['uni.Total Liability scale h2_se']=h2_total['uni.Total Liability scale h2_se']
phenotypes_par_filtered['uni.Total Liability scale h2_z']=phenotypes_par_filtered['uni.Total Liability scale h2']/phenotypes_par_filtered['uni.Total Liability scale h2_se']

In [ ]:
for group_id in list(filter(lambda x: x!=0,phenotypes_par_filtered['group_id'].unique())):
    phenotypes_par_filtered_selected=phenotypes_par_filtered[phenotypes_par_filtered['group_id']==group_id]
    print(phenotypes_par_filtered_selected.sort_values('uni.Total Liability scale h2_z',ascending=False)[['description']])
    drop_idx_list=phenotypes_par_filtered_selected.sort_values('uni.Total Liability scale h2_z',ascending=False).index[1:]
    phenotypes_par_filtered.drop(drop_idx_list,axis='index',inplace=True)

In [16]:
phenotypes_par_filtered.shape

(497, 19)

In [17]:
phenotypes_uni_filtered.loc[phenotypes_par_filtered.index].to_csv(phenotypes_par_filtered_path)